# PROJECT-1 : TWITTER SENTIMENT ANALYSIS


The following project is about analyzing the sentiments of tweets on social networking website‘Twitter’. The dataset for this project  is scraped from Twitter. It contains 1,600,000 tweetsextracted using Twitter API. It is a labeled dataset with tweets annotated with the sentiment (0 =negative,2= neutral,4 = positive)

It contains the following 6 fields:
 1. target: the polarity of the tweet (0= negative,2= neutral,4= positive)
 2. ids: The id of the tweet .
 3. date: The date of the tweet (Sat May 16 23:58:44 UTC2009)
 4. flag: The query. If there is no query, then this value is NO_QUERY.
 5. user: The user that tweeted
 6. text: The text of the tweet.

Design a classification model that correctly predicts the polarity of the tweets provided in the dataset.

### Importing Libraries and Dataset

In [5]:
! pip install contractions


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Importing normal libraries

In [5]:
import pandas as pd
from nltk.corpus import stopwords
import contractions
import re

Importing classifier and ML-related libraries

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [7]:
df = pd.read_csv(r'C:\Users\aasth\OneDrive\Desktop\Guvi_Projects_Module_21\dataset\twitter_new.csv',
                 encoding='ISO-8859-1',header=None)

In [8]:
df.columns=['target','ids','date','flag','user','text']
df

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [9]:
df['text']

0          @switchfoot http://twitpic.com/2y1zl - Awww, t...
1          is upset that he can't update his Facebook by ...
2          @Kenichan I dived many times for the ball. Man...
3            my whole body feels itchy and like its on fire 
4          @nationwideclass no, it's not behaving at all....
                                 ...                        
1599995    Just woke up. Having no school is the best fee...
1599996    TheWDB.com - Very cool to hear old Walt interv...
1599997    Are you ready for your MoJo Makeover? Ask me f...
1599998    Happy 38th Birthday to my boo of alll time!!! ...
1599999    happy #charitytuesday @theNSPCC @SparksCharity...
Name: text, Length: 1600000, dtype: object

### Text Preprocessing

Removing Stop words

In [10]:
stop_words = stopwords.words('english')

In [11]:
df['text'] = df['text'].apply(lambda x : ' '.join([word for word in x.split() if word not in stop_words]))
df['text']

0          @switchfoot http://twitpic.com/2y1zl - Awww, t...
1          upset can't update Facebook texting it... migh...
2          @Kenichan I dived many times ball. Managed sav...
3                           whole body feels itchy like fire
4          @nationwideclass no, behaving all. i'm mad. he...
                                 ...                        
1599995        Just woke up. Having school best feeling ever
1599996    TheWDB.com - Very cool hear old Walt interview...
1599997                 Are ready MoJo Makeover? Ask details
1599998    Happy 38th Birthday boo alll time!!! Tupac Ama...
1599999    happy #charitytuesday @theNSPCC @SparksCharity...
Name: text, Length: 1600000, dtype: object

Removing special characters

In [ ]:
def remove_special_char(x):
    x = x.split()
    for i in range(0,len(x)):
        if "@" in x[i]:
            x[i]= x[i].replace("@","")
        if "#" in x[i]:
            x[i]=x[i].replace("#","")
    return ' '.join(x)

In [ ]:
df['text']= df['text'].apply(lambda x: remove_special_char(x))
df['text']

Removing digits

In [ ]:
#The isdigit() method returns True if all characters in a string are digits. If not, it returns False
def remove_digits(x):
    x = x.split()
    for i in range(0,len(x)):
        if x[i].isdigit():
            x[i]= ""
    return ' '.join(x)

In [ ]:
df['text']=df['text'].apply(lambda x: remove_digits(x))
df['text']

Converting each word in tweets to lowercase

In [ ]:
df['text']=df['text'].apply(lambda x:x.lower())
df['text']

Expanding the contractions

In [ ]:
def expand_contractions(x):
    x = x.split()
    for i in range(0,len(x)):
        x[i] = contractions.fix(x[i])
    return ' '.join(x)

In [ ]:
import contractions
print(contractions.fix("you've"))
print(contractions.fix("he's"))

In [ ]:
df['text'] = df['text'].apply(lambda x:expand_contractions(x))
df['text']

Removal of emails from the tweet

In [ ]:
df1 = df.copy()
df1['emails'] = df1['text'].apply(lambda x: re.findall(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+)', x))
df1['email_count'] = df1['emails'].apply(lambda x: len(x))
df1[df1['email_count'] > 0]

# We can see there are no email_ids present in tweets

Removal of special characters

In [ ]:
df1['special_char'] = df1['text'].apply(lambda x: re.findall(r'[^\w]+', x))
df1['special_char_count'] = df1['special_char'].apply(lambda x: len(x))
df1['special_char_count']

In [ ]:
df['text'] = df['text'].apply(lambda x : re.sub(r'[^\w]+'," ",x))
df['text']

Removal of urls

In [ ]:
pattern = r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?'

In [ ]:
df1['url'] = df1['text'].apply(lambda x: re.findall(pattern, x))
df1['url_count'] = df1['url'].apply(lambda x: len(x))
df1['url_count']

In [ ]:
df['text'] = df['text'].apply(lambda x : re.sub(pattern," ",x))
df['text']